# The Hartree-Fock Method : an example with STO-3G functions for 1s orbitals

This python code aims to illustrate on a simple example how to perform a restrained Hartree-Fock calculation on two examples : dihydrogen and HeH$^+$. It aims to illustrate the algorithmic procedure to build the fock matrix and run a SCF procedure. It was written for students of the ENS Lyon, formation Sciences de la matière, at the L3 level (undergraduates).

It goes with a full course where the notes are available here (in French) :
http://agregationchimie.free.fr/cours.php#HartreeFock

To reach a broader audience, all the equations refer to the book :

* **Modern quantum chemistry : introduction to advanced electronic structure theory**
  *Attila Szabo, Neil S. Ostlund*
  ISBN 9780486691862

This codes is heavily inspired on this page :
https://medium.com/analytics-vidhya/practical-introduction-to-hartree-fock-448fc64c107b
but was written from scratch and takes a slightly different route sometimes.

This code is given under a CC-BY-NC-SA licence. It was written by Martin Vérot, full time teacher at the ENS de Lyon university.

* [Reading a xyz File](#xyz)
* [Reading the Basis Set](#basis-set)
* [Number of Electrons](#nb-elec)
* [Integrals Computation](#calcul-int)
 * [Definition of an Orbital Class](#classe-orb)
 * [Definition of an Orbital Product Class](#classe-prod)
 * [Creation of the Atomic Orbitals $\chi_\mu$](#orb-creation)
 * [Computing the Overlap Matrix](#overlap)
 * [Computing the Kinetic Energy Matrix](#kinetic)
 * [Computing the Nucleus-Electron Potential Matrix](#TNe)
 * [Computing the Electronic Repulsion](#intbi)
* [Initializing the Density Matrix](#densitymatrix)
* [Building up the Bi-electronic Matrix](#intbiG)
* [SCF Procédure](#scf)

In [1]:
#import of libraries
import numpy as np
import scipy
from scipy.special import erf
import json

Definition of some standard variables : elements abbreviations and their atomic number. Here, only H and He will be used.

In [2]:
#Atomes 
AtomList = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']
AtomCharges = np.arange(1,119,1)

<span id="xyz"></span>
# Reading a xyz File

.xyz Files are a common way to share molecular geometries. This function will help to read them automatically. The first line of a .xyz file contains the full number of atoms in the molecule. The second line is a comment. The following lines contain the atomic symbol of each atom followed by its coordinates along x, y, and z.

In [3]:
def read_xyz_file(file):
    """
    Function to read a xyz fil
    - file is the filename where we will read the coordinates, it must respect the .xyz standard
    """
    file = open(file,'r')
    nbAtoms = 0 #number of atoms read 
    check = 0 # nombre of atoms indicated on the first line of the xyz file
    Atoms = [] # List of the atom types
    coordinates = [] #List of the atom coordinates
    #Reading of the file
    for idx, line in enumerate(file):
        #Storing the number of atoms that we should read
        if idx == 0:
            check = int(line.split()[0])
        #The comment line is ignored then we read the coordinates line by line
        elif idx>1:
            split = line.split() #split the line
            Atoms.append(split[0]) #reading the atom type
            #reading the coordinates and converting them to a numpy array of floats
            coordinates.append(np.array([split[1],split[2],split[3]],dtype='float'))
            nbAtoms+=1 #incrementing the number of atoms
    file.close() 
    if check != nbAtoms:
        print('WARNING : the number of coordinates read is different from the theoretical number of atoms. You may have to suppress blank lines at the end of your file.')
    return nbAtoms, Atoms, coordinates

In [4]:
#example files
file = 'H2.xyz'
file = 'HeH.xyz'
#reading of the .xyz file containing the atom coordinate
nbAtoms, Atoms, coordinates = read_xyz_file(file)

We now will create a new array with the atomic charges corresponding to each atom. 
It will be useful to compute the Nucleaus-ELectron potential and the nuclear repulsion.
To this end we use the correspondance of index between `AtomList` and `AtomCharges`. 

In [5]:
AtomicCharges = []
for atom in Atoms:
    idx = AtomList.index(atom)
    AtomicCharges.append(AtomCharges[idx])

<span id="basis-set"></span>
# Reading the Basis Set
We will now read the basis set assuming a minimal STO-3G basis set. The code should work for different basis sets *as long as they are based on gaussian integrals and contain only s orbitals*.

The file used by default is a modification from the file obtained from https://www.basissetexchange.org/ with a json export.

In [6]:
def readBasisSetJSON(file):
    """
    Reading the basis set 
    - file must contain the basis set in a json file similar to the one 
      produced by the https://www.basissetexchange.org/ website
    WARNING :
    - the basis set corresponding to coefficients given in Szabo p159 correspond to the "version 0" of the site, 
      to get this version you must use the "advanced mode"
    - Simlarly, the coefficient given for Helium are not the same as the ones given in Szabo p170.
    - this program does NOT WORK with p orbitals, 
      ***it is ONLY possible to perform calculations on molecules containing ONLY H and He atoms !***
    - To get a proper correspondance between basis sets and atoms, the file must contain 
      all the basis sets for all atoms from hydrogen up to the heaviest one. 
    """ 
    f = open(file_basis)
    basis_sets = json.load(f)
    #Exponents and coefficients for s orbitals
    Sexponents = []
    Scoefficients = []
    #Exponents and coefficients for p orbitals
    Pexponents = []
    Pcoefficients = []

    #For each element, we read the basis set:
    for i,element in enumerate(basis_sets['elements']):
        #reading for each subshell
        #print(AtomList[int(element)-1]) #printing the element
        #Adding a set of exponents and coefficients for each element.
        Sexponents.append([])
        Pexponents.append([])
        Scoefficients.append([])
        Pcoefficients.append([])
        """
        Then we read the file by ascending order, it shoulf fill naturally all the subshells.
        This part should not work for element with d orbitals (Z>=21, Scandium)
        """
        for shell in basis_sets['elements'][element]['electron_shells']: 
            #reading the exponents and coefficients for each subshell
            for subshell in shell['angular_momentum']:
                AtomExponents = np.asarray(shell['exponents'],dtype="float")
                AtomCoefficients = np.asarray(shell['coefficients'][subshell],dtype="float")
                if subshell == 0:
                    Sexponents[i].append(AtomExponents)
                    Scoefficients[i].append(AtomCoefficients)
                if subshell == 1:
                    Pexponents[i].append(AtomExponents)
                    Pcoefficients[i].append(AtomCoefficients)
    return Sexponents,Scoefficients,Pexponents,Pcoefficients

In [7]:
#file containing the basis set
file_basis = 'sto-3g0.json' #Szabo coefficients for H and website ones for He
file_basis = 'sto-3gExchange.json' #website coefficients for H and He
file_basis = 'sto-3gSzabo.json' #Szabo coefficients for H and He
Sexponents,Scoefficients,Pexponents,Pcoefficients = readBasisSetJSON(file_basis)

In [8]:
#Printing the exponents and coefficients for each primitive gaussian of each orbital.
for i,coeffs in enumerate(Sexponents):
    print(AtomList[i])
    for j,exponents in enumerate(coeffs):
        print('{}s'.format(j+1))
        print(exponents)
        print(Scoefficients[i][j])

H
1s
[3.42525091 0.62391373 0.1688554 ]
[0.15432897 0.53532814 0.44463454]
He
1s
[9.75393462 1.77669115 0.48084429]
[0.15432897 0.53532814 0.44463454]


<span id="nb-elec"></span>
# Number of Electrons

In [9]:
NbElectrons = int(2)
#Number of doubly occupied orbitals (this code works within the RESTRAINED Hartree-Fock formalism)
orbOccupied = int(NbElectrons/2)

<span id="calcul-int"></span>
# Integrals Computation

<span id="classe-orb"></span>
### Definition of an Orbital Class


WARNING :
$\chi^\mathrm{G}_{\alpha_{p\mu},\mathbf{R_A}}\left(\mathbf{r} \right) = \left(\dfrac{2 \alpha_{p\mu}}{\pi}\right)^{0.75}\exp(-\alpha_{p\mu} |\mathbf{r}-\mathbf{r}_A|^2)$

We will need to store orbitals. Instead of computing directly the full orbital in its complete form :
$\chi^{CGF}_{\mu}\left(\mathbf{r}-\mathbf{R}_A \right) = \sum_{p=1}^{L} c_{p\mu}\chi^\mathrm{G}_{\alpha_{p\mu},\mathbf{R_A}}\left(\mathbf{r} \right)$
we will store it with two arrays :
* an array with the gaussian exponents $\{\alpha_{p\mu}\}$
* an array with the contraction coefficients $\{c_{p\mu}\}$ 
* and an array containing the coordinates of the center of the orbital
When we will have to make gaussian products, it will be much easier to compute them this way to use some results about integrals of gaussian functions.

We also give a human readable name for the orbital from : 
- the indix of the atom in the `Atoms` list
- the symbol of the element
- the corresponding shell

Thus, for HeH$^+$ the first orbital is called `0He1s` and the second `1H1s`. 

In [10]:
class Orbital:
    """
    name : the orbital name : number of the atom, its symbol, then the orbital type
    center : 3D coordinates of the atom bearing the orbital
    exponents :  exponents of the gaussians, stored as a numpy array
    coeffs : coefficients of the gaussians, stored as a numpy array
    """
    def __init__(self, name, center, exponents, coeffs):
        self.name = name
        self.center = center
        self.exponents = exponents
        self.coeffs = coeffs
    def __repr__(self):
        return 'name :         {}\ncenter :       {}\nexponents    : {}\ncoefficients : {}'.format(self.name,self.center,self.exponents,self.coeffs)

<span id="classe-prod"></span>
### Definition of an Orbital Product Class
Similarly, we create a new Class corresponding to a gaussian product. To this end, we store :
- `p` the gaussian exponents which are the sum of the original exponants ($p = \alpha+ \beta$ Szabo, equation 3.209 p 154)
- `ab` the product of the two exponents (useful to compute some integrals later) ($ab=\alpha\times \beta$)
- `diffR` the norm of the difference between the two centers of each original gaussian ($|\mathbf{r}_A-\mathbf{r}_B|^2$)
- `K` the normalisartion constante corresponding to $\mathtt{K}= \exp(-\mathtt{ab}/\mathtt{p}*\mathtt{diffR})\times \left(\dfrac{4 \alpha\times \beta }{\pi^2}\right)^{0.75} \times c_A\times c_B $ (Szabo, equation 3.208 p154 with an additional factor of $\left(\dfrac{4 \alpha\times \beta }{\pi^2}\right)^{0.75}$ to take into account the normalization of the gaussians and the contraction coefficients $c_A$ et $c_B$ which are the $c_{p\mu}$.)
- `Rp` the coordinates of the new center of the gaussian product

We thus have $\chi_\mu\times\chi_\nu = \sum_i K_i\exp(-p_i |\mathbf{r}-\mathbf{Rp}|^2)$

In [11]:
class OrbitalProduct:
    """
    name : name of the product
    p : exponents linked to each term of the product
    diffR : norm of the difference between orbital centers
    K : normalized coefficient linked to each term of the product
    Rp : center of the gaussian product
    """
    def __init__(self,name,p, ab,diffR, K, Rp):
        self.name = name
        self.p = p
        self.ab = ab
        self.diffR = diffR
        self.K = K
        self.Rp = Rp

We can now use the results given p154 or 411 of the Szabo to perform the calculation on the product of two gaussians.

In [12]:
#Computation of the product of gaussians (Szabo p411)
#The N factor gives the proper factor to normalize the product.
def ProductGaussians(orba,orbb,i,j):
    #sum of the exponents
    p = orba.exponents[i] + orbb.exponents[j]
    ab = orba.exponents[i]*orbb.exponents[j]
    #computation of the norm of the difference between gaussian centers
    diffR = (np.linalg.norm(orba.center-orbb.center))**2
    #Normalization coefficient
    N = (4*ab/(np.pi**2))**0.75
    K = N*np.exp(-ab/p*diffR)*orba.coeffs[i]*orbb.coeffs[j]
    #center of the gaussian product
    Rp = (orba.exponents[i]*orba.center+orbb.exponents[j]*orbb.center)/p
    return p,ab, diffR, K, Rp

Then we can generalize the computation of gaussian products which are the sum of gaussians. The product of two STO-3G gaussians gives rise to 9 new gaussian orbitals. 

In [13]:
def ProductOrbitals(orba,orbb):
    ps=[]
    Ks=[]
    Rps=[]
    abss=[]
    for i in range(orba.exponents.shape[0]):
        for j in range(orbb.exponents.shape[0]):
            p, ab, diffR, K, Rp = ProductGaussians(orba,orbb,i,j)
            ps.append(p)
            Ks.append(K)
            Rps.append(Rp)
            abss.append(ab)
    return OrbitalProduct('{}x{}'.format(orba.name,orbb.name),np.asarray(ps),np.asarray(abss),diffR,np.asarray(Ks),np.asarray(Rps))

<span id="orb-creation"></span>
## Creation of the Atomic Orbitals $\chi_\mu$
For each atom, we create a set of orbitals based on the basis set. For that, we use the `Orbital` class defined [here](#classe-orb).

`nbOrbitals` is the total number of atomic orbitals (written as $M$ in the notes)

In [14]:
Orbitals = []
#total number of atomic orbitals
nbOrbitals = 0

"""
Assigning a set of orbitals to each atom : we create a et of orbitals centered on each atom.
"""
for i,atom in enumerate(Atoms):
    idx = AtomList.index(atom)
    for j,exponents in enumerate(Sexponents[idx]):
        Orbitals.append(Orbital('{}{}{}s'.format(i,atom,j+1),coordinates[i],exponents,Scoefficients[idx][j]))
        nbOrbitals +=1    

In [15]:
#Printing of each orbital created        
for i,orb in enumerate(Orbitals):
    print(orb)

name :         0He1s
center :       [0. 0. 0.]
exponents    : [9.75393462 1.77669115 0.48084429]
coefficients : [0.15432897 0.53532814 0.44463454]
name :         1H1s
center :       [0.     0.     1.4632]
exponents    : [3.42525091 0.62391373 0.1688554 ]
coefficients : [0.15432897 0.53532814 0.44463454]


<span id="overlap"></span>
## Computing the Overlap Matrix
We use formula A.9 p412 of Szabo with an adapted version taking into account the coefficients of the orbital and the normalisation factors for `K`.

In [16]:
def Overlap(gp):
    """
    Computing the overlap : the integral of the gaussian product. 
    The commented formula is equivalent to the one used below.
    - gp is an `OrbitalProduct` object.
    """
    #S = 0
    #for j in range(len(gp.p)):
    #    S+=gp.K[j]*(np.pi/gp.p[j])**1.5
    S = np.sum(gp.K*(np.pi/gp.p)**1.5)
    return S

In [17]:
#Creation of the overlap matrix
OverlapMatrix = np.zeros((nbOrbitals,nbOrbitals))

#Computing each term of the matrix. As S is here a real symmetric matrix, only half of it is computed.
for i in range(nbOrbitals):
    for j in range(i,nbOrbitals):
        product = ProductOrbitals(Orbitals[i],Orbitals[j])
        S = Overlap(product)
        OverlapMatrix[i,j]=S
        OverlapMatrix[j,i]=S

print(OverlapMatrix)

[[1.00000085 0.45076978]
 [0.45076978 0.99999999]]


<span id="kinetic"></span>
## Computing the Kinetic Energy Matrix
We use formula A.11 p412 of Szabo.

In [18]:
def Kinetic(gp):
    """
    Computing the kinetic energy.
    The commented formula is equivalent to the one used below.
    - gp is an `OrbitalProduct` object.
    """
    #K = 0
    #for j in range(len(gp.p)):
    #    prefactor = gp.ab[j]/gp.p[j]*(3-2*gp.ab[j]/gp.p[j] *gp.diffR )
    #    K+=prefactor*gp.K[j]*(np.pi/gp.p[j])**1.5
    return np.sum(gp.ab/gp.p *(3-2*gp.ab/gp.p*gp.diffR)*gp.K*(np.pi/gp.p)**1.5)   

In [19]:
#Creating the kinetic matrix
KineticMatrix = np.zeros((nbOrbitals,nbOrbitals))

#Computing each term of the matrix. As K is here a real symmetric matrix, only half of it is computed.
for i in range(nbOrbitals):
    for j in range(i,nbOrbitals):
        product = ProductOrbitals(Orbitals[i],Orbitals[j])
        K = Kinetic(product)
        KineticMatrix[i,j]=K
        KineticMatrix[j,i]=K

print(KineticMatrix)

[[2.16431219 0.16701245]
 [0.16701245 0.76003188]]


<span id="TNe"></span>
## Computing the Nucleus-Electron Potential Matrix
We use formula A.33 p415 of Szabo. 

In [20]:
#Function defined equation A.32 p415 of Szabo 
"""
(Boys function, more information is available on them in 
Molecular Electronic-Structure Theory
  Trygve Helgaker, Poul Jorgensen, Jeppe Olsen
  2013, Wiley
"""
def Fo(t):
    if t==0:
        return 1
    else:
        return (0.5*(np.pi/t)**0.5)*erf(t**0.5)
    
def Potential(gp):
    """
    Computation of V_Ne
    The commented formula is equivalent to the one used below.
    - gp is an `OrbitalProduct` object.
    """
    V = 0
    for j in range(len(gp.p)):
        #On somme les termes en Z/rAi :
        for idx,Z in enumerate(AtomicCharges):
            prefactor = -2*np.pi/gp.p[j]*Z*Fo(gp.p[j]*(np.linalg.norm(gp.Rp[j]-coordinates[idx]))**2)
            V+=prefactor*gp.K[j]
    return V   

In [21]:
#creating the potential energy matrix
VMatrix = np.zeros((nbOrbitals,nbOrbitals))

#Computing each term of the matrix. As V is here a real symmetric matrix, only half of it is computed.
for i in range(nbOrbitals):
    for j in range(i,nbOrbitals):
        product = ProductOrbitals(Orbitals[i],Orbitals[j])
        V = Potential(product)
        VMatrix[i,j]=V
        VMatrix[j,i]=V

print(VMatrix)

[[-4.81705533 -1.51421594]
 [-1.51421594 -2.49185753]]


## Computing the Mono-Electronic part $H^\mathrm{core}$
Using equation 3.153 p141 of Szabo

In [22]:
HcoreMatrix = KineticMatrix+VMatrix
print(HcoreMatrix)

[[-2.65274315 -1.34720349]
 [-1.34720349 -1.73182566]]


<span id="intbi"></span>
## Computing the Electronic Repulsion
We use formula A.41 p 416 of Szabo

In [23]:
def integralbi(orba,orbb,orbc,orbd):
    #a and b are occupied by electron 1
    #c and d are occupied by electron 2
    prAB = ProductOrbitals(orba,orbb)
    prCD = ProductOrbitals(orbc,orbd)
    I=0
    for i in range(len(prAB.p)):
        for j in range(len(prCD.p)):
            prefactor = (2*np.pi**2.5)/(prAB.p[i]*prCD.p[j]*(prAB.p[i]+prCD.p[j])**0.5)*Fo(prAB.p[i]*prCD.p[j]/(prAB.p[i]+prCD.p[j])*(np.linalg.norm(prAB.Rp[i]-prCD.Rp[j]))**2)
            I+=prefactor*prAB.K[i]*prCD.K[j]
    return I 

In [24]:
"""
Checking the values given p162 equation 3.235 of Szabo
combi = (0,1,0,1)
print(combi)
print(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))
combi = (0,0,1,1)
print(combi)
print(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))
combi = (0,0,0,0)
print(combi)
print(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))
combi = (1,1,1,1)
print(combi)
print(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))
"""

'\nChecking the values given p162 equation 3.235 of Szabo\ncombi = (0,1,0,1)\nprint(combi)\nprint(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))\ncombi = (0,0,1,1)\nprint(combi)\nprint(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))\ncombi = (0,0,0,0)\nprint(combi)\nprint(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))\ncombi = (1,1,1,1)\nprint(combi)\nprint(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))\n'

 <span id="densitymatrix"></span>
 # Initializing the Density Matrix

In [25]:
def diagonalizeAscendingEigenvalues(M):
    """
    Diagonalizing the M matrix with eigenvalues in ascending order.
    Important to build the density matrix as it uses only occupied orbitals.
    """
    eigVal,eigVec = np.linalg.eig(M)
    idx = np.argsort(eigVal)
    eigVal = eigVal[idx]
    eigVec = eigVec[:,idx]
    return eigVal,eigVec

In [26]:
def buildPmatrix(Cmatrix,nbOrbitals,orbOccupied):
    """
    Building the density matrix from
    - the Cmatrix coefficients containing all the decomposition of the orbitals on the atomic orbitals
    - the full number of orbitals nbOrbitals
    - orbOccupied which is the number of **occupied orbitals**.
    See equation 3.145 p139 of Szabo
    """
    Pmatrix = np.zeros_like(Cmatrix)
    for i in range(nbOrbitals):
        for j in range(nbOrbitals):
            Pmatrix[i][j]=2*np.sum(Cmatrix[i,0:orbOccupied]*Cmatrix[j,0:orbOccupied])
            Pmatrix[j][i]=Pmatrix[i][j]
    return Pmatrix

For our first guess for the density matrix, we diagonalize $H^\mathrm{core}$.

In [27]:
E0,C0 = diagonalizeAscendingEigenvalues(HcoreMatrix)
Cmatrix = C0
print(Cmatrix)

#Building the density matrix from the coefficient matrix
Pmatrix = buildPmatrix(Cmatrix,nbOrbitals,orbOccupied)

[[-0.8134554   0.58162729]
 [-0.58162729 -0.8134554 ]]


<span id="intbiG"></span>
# Building up the Bi-electronic Matrix
equation 3.154 p141 of Szabo

In [28]:
def BuildBiMatrix(nbOrbitals,Pmatrix,Orbitals):
    """
    Integralbi uses the "chemists" notation : integralbi(a,b,c,d) = (ab|cd) (chemist notation) = <ac||bd> "physicist notation"
    - nbOrbitals number of orbitals 
    - Pmatrix density matrix
    - Orbitals : atomic orbitals
    """
    BiMatrix = np.zeros((nbOrbitals,nbOrbitals))
    for mu in range(nbOrbitals):
        for nu in range(nbOrbitals):
            G = 0
            for sigma in range(nbOrbitals):
                for llambda in range(nbOrbitals):
                    G += Pmatrix[sigma][llambda]*(integralbi(Orbitals[mu],Orbitals[nu],Orbitals[sigma],Orbitals[llambda])-0.5*integralbi(Orbitals[mu],Orbitals[llambda],Orbitals[sigma],Orbitals[nu]))
            BiMatrix[mu,nu]=G
            BiMatrix[nu,mu]=G
    return BiMatrix

<span id="symmbase"></span>
# Orthonormalization of the basis set
Voir p 142-143 du Szabo.

In [29]:
#diagonalization of the overlap matrix; equation 3.166 p143 of Szabo.
evalS,U = np.linalg.eig(OverlapMatrix)
"""
U is the  transition matrix such that s is diagonal if we compute 
U.T × OverlapMatrix × U = s with s diagonal
"""
print(U)
#We take the eigenvalues, we place them on the diagonal and take the inverse of the square root
invsqrtS = np.diag(evalS**-0.5)
#equation 3.167 of Szabo
matZ = np.dot(U,np.dot(invsqrtS,U.T))
print(matZ)
#checking equation (3.168) to show that the basis set is made orthoganl with the use of matZ
#print(np.dot(matZ.T,np.dot(OverlapMatrix,matZ)))

[[ 0.70710712 -0.70710645]
 [ 0.70710645  0.70710712]]
[[ 1.08978882 -0.25955474]
 [-0.25955474  1.08978932]]


<span id="scf"></span>
# SCF Procedure

## Convergence Criteria
We check if the matrix density has converged compared to the previous iteration. We couls also take a criteria based on the energy, or on both.

In [30]:
def diffP(P1,P2):
    """
    - P1 density matrix at the previous step
    - P2 density matrix at this step
    """
    return np.sqrt(np.sum((P1-P2)**2))

## Computing the nuclear energy

In [31]:
def Enuc(AtomicCharges,coordinates):
    """
    AtomicCharges : Agomic number of each atom
    coordinates : nuclear coordinates
    """
    E = 0
    for A,ZA in enumerate(AtomicCharges):
        for B,ZB in enumerate(AtomicCharges):
            if A != B: #On ne veut pas calculer la répulsion d'un atome avec lui même
                E+=ZA*ZB/np.linalg.norm(coordinates[A]-coordinates[B])
    return 0.5 * E #facteur 0.5 pour éviter le double comptage

## SCF Procedure
The different steps are the one given p 146 of Szabo.

In [32]:
# Number of iterations performed , we stop if there is no convergence after 100 iterations
step = 1
#Initializing the convergence criteria 
Threshold = 100
while Threshold > 1e-8 and step < 100:
    #Construction of the bi-electronic part (step 5)
    BiMatrix = BuildBiMatrix(nbOrbitals,Pmatrix,Orbitals)
    #print(BiMatrix)
    #Building the matrix of the Fock operator (step 6)
    Fmatrix = HcoreMatrix + BiMatrix
    #Building the F' matrix  in the orthogonal basis (step 7)
    FmatrixPrime = np.dot(matZ.T,np.dot(Fmatrix,matZ))
    #Diagonalization of F' (step 8)
    E,Cprime = diagonalizeAscendingEigenvalues(FmatrixPrime)
    #Computation of the coefficients in the initial basis set (step 9)
    Cmatrix = np.dot(matZ,Cprime)
    #Computation of the total energie (eq 3.274 p176)
    Etot = 0.5*np.sum(Pmatrix*(HcoreMatrix+Fmatrix))
    Efull = Etot+Enuc(AtomicCharges,coordinates)
    #Storing the previous density matrix
    oldP = Pmatrix.copy()
    #Computing the new density matrix (step 10)
    Pmatrix = buildPmatrix(Cmatrix,nbOrbitals,orbOccupied)
    #Computing the convergence criteria (step 11)
    Threshold = diffP(oldP,Pmatrix)
    print('step {}'.format(step))
    print('Orbitalar Energies')
    print(E)
    print('Electronic E \t {}\ntotal E \t {}'.format(Etot,Efull))
    print('S','F')
    print(np.hstack((OverlapMatrix,Fmatrix)))
    print('C','P')
    print(np.hstack((Cmatrix,Pmatrix)))
    print('\n')
    step+=1


step 1
Orbitalar Energies
[-1.25593539  0.47218814]
Electronic E 	 -5.39390058663111
total E 	 -4.027033446117168
S F
[[ 1.00000085  0.45076978 -1.02417307 -0.9873405 ]
 [ 0.45076978  0.99999999 -0.9873405  -0.49044868]]
C P
[[-0.74559801  0.83611719  1.11183279  0.61177688]
 [-0.41025919 -1.04244751  0.61177688  0.33662521]]


step 2
Orbitalar Energies
[-1.62975556 -0.06681464]
Electronic E 	 -4.213510495121946
total E 	 -2.8466433546080046
S F
[[ 1.00000085  0.45076978 -1.48450154 -1.07424023]
 [ 0.45076978  0.99999999 -1.07424023 -0.8358076 ]]
C P
[[-0.79845059  0.78580214  1.27504668  0.54537623]
 [-0.34152159 -1.06694537  0.54537623  0.233274  ]]


step 3
Orbitalar Energies
[-1.59958252 -0.06184633]
Electronic E 	 -4.227470149187212
total E 	 -2.8606030086732703
S F
[[ 1.00000085  0.45076978 -1.46030425 -1.05221382]
 [ 0.45076978  0.99999999 -1.05221382 -0.81214633]]
C P
[[-0.80165974  0.78252796  1.28531667  0.54056291]
 [-0.33715234 -1.06833409  0.54056291  0.2273434 ]]


step 4